In [1]:
import json
import os 
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2
import numpy as np
from create_dataset_json import *
import random

In [2]:
data_root = "D:\\DeepLearningFiles\\renders_multicam_diff_all"
output_path = "C:\\Users\\Ugne\\Documents\\studies\\Python\\DLGroupTask\\synthetic_anno.json"

In [3]:
anno_dict = dict({"images": [], "annotations": [], "categories": []})
files = find_files(data_root)
all_scenes = [get_scene_name_from_file(file, data_root) for file in files]

# sampling a 100 scenes for the first attempt

selected_scenes = random.sample(all_scenes, 500)
selected_files = [file for file in files if any(scene in file["label1"] for scene in selected_scenes)]

In [4]:
len(all_scenes)

1860

In [5]:

    # i is index into images, j index into annotations, k index into categories
    i, j, k = 0, 0, 0
    skus = set()
    for file in selected_files:
        print(file)
        scene_name = get_scene_name_from_file(file, data_root)
        # (From preprocess_synthetic_data.py)
        label_file1 = file["label1"]
        label_file2 = file["label2"]
        label1 = cv2.imread(label_file1, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)[:, :, 0]
        label2 = cv2.imread(label_file2, cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)[:, :, 0]
        label1_json = file["label1_json"]
        with open(label1_json) as f:
            label1_json = json.load(f)
        label2_json = file["label2_json"]
        with open(label2_json) as f:
            label2_json = json.load(f)
        items1_removed = set(label1_json["removed_skus"]) # Should be empty
        items1_added = set(label1_json["added_skus"]) # Should be empty
        items2_removed = set(label2_json["removed_skus"])
        items2_added = set(label2_json["added_skus"])
        items2_shifted = set(label2_json["shifted_skus"])
        bounding_boxes = make_bbox_camogram(
            file["label1_json"], file["label2_json"], file["label1"], file["label2"]
        )
        change_mask_added = np.zeros_like(label1)
        change_mask_removed = np.zeros_like(label1)
        change_mask_shifted = np.zeros_like(label1)
        change_mask_added = make_masks(change_mask_added, items1_removed, label2_json, label2, "put")
        change_mask_removed = make_masks(change_mask_removed, items1_added, label1_json, label1, "take")
        change_mask_removed = make_masks(
            change_mask_removed, items2_removed, label1_json, label1, "take"
        )
        change_mask_added = make_masks(change_mask_added, items2_added, label2_json, label2, "put")
        change_mask_shifted = make_masks(
            change_mask_shifted,
            items2_shifted,
            label1_json,
            label1,
            "shift",
            label2_json=label2_json,
            label2=label2,
        )
        change_mask = np.dstack((change_mask_removed, change_mask_added, change_mask_shifted))

        # Create dict entry in images list
        height, width = label1.shape
        image1_name = scene_name+"_change-0.png"
        image2_name = scene_name+"_change-1.png"
        randommat1_name = scene_name+"_change-0-randommats.png"
        randommat2_name = scene_name+"_change-1-randommats.png"
        depth1_name = scene_name+"_change-0-depth0001.exr"
        depth2_name = scene_name+"_change-1-depth0001.exr"
        image_dict = dict({
            "id": i, 
            "license": 1,
            "scene": scene_name,
            "width": width, 
            "height": height, 
            "randommats1": randommat1_name,
            "randommats2": randommat2_name,
            "depth1": depth1_name, 
            "depth2": depth2_name, 
            "image1": image1_name, 
            "image2": image2_name})
        anno_dict["images"].append(image_dict)

        # Make a mask for every added SKU
        for item in items2_added:
            change_mask_added = np.zeros_like(label1)
            change_mask_added = make_masks(change_mask_added, set({item}), label2_json, label2, "put")
            if np.any(change_mask_added):
                submasks = create_submask_from_array(change_mask_added)
                for color, sub_mask in submasks.items():
                    if item not in skus:
                        skus.add(item)
                        anno_dict["categories"].append({"id": k, "name": item})
                        sku_id = k
                        k+=1
                    else:
                        sku_id = [d.get("id") for d in anno_dict["categories"] if d["name"] == item][0]
                    anno = create_sub_mask_annotation(np.array(sub_mask), image_id=i, category_id=sku_id, annotation_id=j, is_crowd=0, scene=scene_name)
                    if anno:
                        anno["action"] = "put"
                        anno_dict["annotations"].append(anno)
                        print("put ", sku_id)
                        j+=1
        # Make a mask for every removed SKU
        for item in items2_removed:
            change_mask_removed = np.zeros_like(label1)
            change_mask_removed = make_masks(change_mask_removed, set({item}), label2_json, label2, "put")
            if np.any(change_mask_removed):
                submasks = create_submask_from_array(change_mask_removed)
                for color, sub_mask in submasks.items():
                    if item not in skus:
                        skus.add(item)
                        anno_dict["categories"].append({"id": k, "name": item})
                        sku_id = k
                        k+=1
                    else:
                        sku_id = [d.get("id") for d in anno_dict["categories"] if d["name"] == item][0]
                    anno = create_sub_mask_annotation(np.array(sub_mask), image_id=i, category_id=sku_id, annotation_id=j, is_crowd=0, scene=scene_name)
                    if anno:
                        anno["action"] = "take"
                        anno_dict["annotations"].append(anno)
                        print("take ", sku_id)
                        j+=1
        # Make a mask for every shifted SKU
        for item in items2_shifted:
            change_mask_shifted = np.zeros_like(label1)
            change_mask_shifted = make_masks(change_mask_shifted, set({item}), label2_json, label2, "put")
            if np.any(change_mask_shifted):
                submasks = create_submask_from_array(change_mask_shifted)
                for color, sub_mask in submasks.items():
                    if item not in skus:
                        skus.add(item)
                        anno_dict["categories"].append({"id": k, "name": item})
                        sku_id = k
                        k+=1
                    else:
                        sku_id = [d.get("id") for d in anno_dict["categories"] if d["name"] == item][0]
                    anno = create_sub_mask_annotation(np.array(sub_mask), image_id=i, category_id=sku_id, annotation_id=j, is_crowd=0, scene=scene_name)
                    if anno:
                        anno["action"] = "shift"
                        anno_dict["annotations"].append(anno)
                        print("shift ", sku_id)
                        j+=1
        i+=1

{'label1': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2_change-0-segmentation0001.exr', 'label2': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2_change-1-segmentation0001.exr', 'label': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2-label.png', 'label1_json': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2_change-0_label.json', 'label2_json': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2_change-1_label.json', 'bbox_json': 'D:\\DeepLearningFiles\\renders_multicam_diff_all\\circ.us.0000_0017c0b964c2492db349f0591c6af20a_cam-2-boxes.json'}


TypeError: get_scene_name_from_file() missing 1 required positional argument: 'data_root'

In [ ]:
anno_dict["info"] = dict({
        "description": "Synthetic dataset created in Blender for change detection, instance segmentation, and depth estimation.",
        "url": "https://github.com/Standard-Cognition/blender-synth",
        "version": 1,
        "year": 2021,
        "contributor": "Cristina Mata, Nick Locascio, Mohammed Sheikh, Kenneth Kihara",
        "date_created": "July 1, 2021"
    })
    # Add license
    
anno_dict["licenses"] = [dict({
        "url": "url_to_our_license",
        "id": 1,
        "name": "Attribution License"
    })]


In [ ]:
with open(output_path, 'w') as jsonFile:
    json.dump(anno_dict, jsonFile)